# Trainer

> Scripts to train the models

In [1]:
#| default_exp trainer

In [2]:
#| export
#| hide
import torch
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning import Trainer
from sklearn.model_selection import train_test_split

In [3]:
#| export
def train_model(model, data, labels, config):
    """
    Trains the given model using the provided data and configuration.

    Parameters:
    -----------
    model : pl.LightningModule
        The model to be trained.
    
    data : torch.Tensor or np.array
        The input data (features) to train the model.
    
    labels : torch.Tensor or np.array
        The corresponding labels for the input data.
    
    config : dict
        Configuration dictionary containing training parameters like batch_size, epochs, val_split, etc.
    """
    
    # Split the data and create dataloaders
    if config['val_split'] > 0:
        X_train, X_val, y_train, y_val = train_test_split(
            data, labels,
            test_size=config['val_split'],
            random_state=42
        )
        train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
        val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
        val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'])
    else:
        train_dataset = TensorDataset(torch.tensor(data), torch.tensor(labels))
        val_dataloader = None

    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)

    # Initialize PyTorch Lightning Trainer
    trainer = Trainer(
        max_epochs=config['epochs'],
        log_every_n_steps=10,
        gpus=1 if torch.cuda.is_available() else 0
    )

    # Train the model
    if val_dataloader:
        trainer.fit(model, train_dataloader, val_dataloader)
    else:
        trainer.fit(model, train_dataloader)

    return model.metrics

In [4]:
#| export
def train_model_with_explicit_validation(model, train_data, train_labels, val_data, val_labels, config):
    """
    Trains the given model using the provided data and configuration, with explicitly set validation data.

    Parameters:
    -----------
    model : pl.LightningModule
        The model to be trained.
    
    train_data : torch.Tensor or np.array
        The input data (features) for training.
    
    train_labels : torch.Tensor or np.array
        The corresponding labels for the training data.
    
    val_data : torch.Tensor or np.array
        The input data (features) for validation.
    
    val_labels : torch.Tensor or np.array
        The corresponding labels for the validation data.
    
    config : dict
        Configuration dictionary containing training parameters like batch_size, epochs, etc.
    """

    # Create PyTorch datasets
    train_dataset = TensorDataset(torch.tensor(train_data), torch.tensor(train_labels))
    val_dataset = TensorDataset(torch.tensor(val_data), torch.tensor(val_labels))

    # Create DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'])

    # Initialize PyTorch Lightning Trainer
    trainer = Trainer(
        max_epochs=config['epochs'],
        log_every_n_steps=10,
        gpus=1 if torch.cuda.is_available() else 0
    )

    # Train the model
    trainer.fit(model, train_dataloader, val_dataloader)

    # Optionally return metrics (assuming they are logged within the model)
    return model.metrics

In [5]:
#| hide
import nbdev; nbdev.nbdev_export()